## Reading Data and Importing Libraries ##

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
nrows = 200 * 100000

train = pd.read_csv('riiid_data/train.csv',
                   usecols=[1, 2, 3, 4, 5, 7, 8, 9],
                    nrows = nrows,
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [3]:
#reading in question df
questions_df = pd.read_csv('riiid_data/questions.csv',   
                            nrows = nrows,
                            usecols=[0, 3],
                            dtype={'question_id': 'int16',
                              'part': 'int8'}
                          )

In [4]:
#reading in lecture df
lectures_df = pd.read_csv('riiid_data/lectures.csv')

In [5]:
lectures_df['type_of'] = lectures_df['type_of'].replace('solving question', 'solving_question')

lectures_df = pd.get_dummies(lectures_df, columns=['part', 'type_of'])

part_lectures_columns = [column for column in lectures_df.columns if column.startswith('part')]

types_of_lectures_columns = [column for column in lectures_df.columns if column.startswith('type_of_')]

In [6]:
lectures_df.head()

,lecture_id,tag,part_1,part_2,part_3,part_4,part_5,part_6,part_7,type_of_concept,type_of_intention,type_of_solving_question,type_of_starter
0,89,159,0,0,0,0,1,0,0,1,0,0,0
1,100,70,1,0,0,0,0,0,0,1,0,0,0
2,185,45,0,0,0,0,0,1,0,1,0,0,0
3,192,79,0,0,0,0,1,0,0,0,0,1,0
4,317,156,0,0,0,0,1,0,0,0,0,1,0


In [7]:
# merge lecture features to train dataset
train_lectures = train[train.content_type_id == True].merge(lectures_df, left_on='content_id', right_on='lecture_id', how='left')

In [8]:
train_lectures.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,lecture_id,tag,...,part_2,part_3,part_4,part_5,part_6,part_7,type_of_concept,type_of_intention,type_of_solving_question,type_of_starter
0,653762,2746,6808,1,14,-1,NaN,False,6808,129,...,1,0,0,0,0,0,0,1,0,0
1,10183847,5382,16736,1,21,-1,NaN,False,16736,40,...,0,0,0,0,0,0,1,0,0,0
2,1424348597,5382,30207,1,104,-1,NaN,False,30207,43,...,0,0,0,1,0,0,1,0,0,0
3,1425557777,5382,18545,1,121,-1,NaN,False,18545,58,...,0,0,0,1,0,0,1,0,0,0
4,405813029,8623,10540,1,59,-1,NaN,False,10540,99,...,0,0,0,0,0,0,1,0,0,0


In [9]:
# collect per user stats
user_lecture_stats_part = train_lectures.groupby('user_id')[part_lectures_columns + types_of_lectures_columns].sum()

In [10]:
user_lecture_stats_part.head()

,part_1,part_2,part_3,part_4,part_5,part_6,part_7,type_of_concept,type_of_intention,type_of_solving_question,type_of_starter
user_id,,,,,,,,,,,
2746,0,1,0,0,0,0,0,0,1,0,0
5382,1,0,0,0,2,0,0,3,0,0,0
8623,2,1,0,0,0,0,0,3,0,0,0
12741,0,0,0,3,0,1,2,4,0,2,0
13134,1,3,0,0,3,0,0,6,1,0,0


In [11]:
# add boolean features
for column in user_lecture_stats_part.columns:
    bool_column = column + '_boolean'
    user_lecture_stats_part[bool_column] = (user_lecture_stats_part[column] > 0).astype(int)

In [12]:
user_lecture_stats_part.head()

,part_1,part_2,part_3,part_4,part_5,part_6,part_7,type_of_concept,type_of_intention,type_of_solving_question,...,part_2_boolean,part_3_boolean,part_4_boolean,part_5_boolean,part_6_boolean,part_7_boolean,type_of_concept_boolean,type_of_intention_boolean,type_of_solving_question_boolean,type_of_starter_boolean
user_id,,,,,,,,,,,,,,,,,,,,,
2746,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
5382,1,0,0,0,2,0,0,3,0,0,...,0,0,0,1,0,0,1,0,0,0
8623,2,1,0,0,0,0,0,3,0,0,...,1,0,0,0,0,0,1,0,0,0
12741,0,0,0,3,0,1,2,4,0,2,...,0,0,1,0,1,1,1,0,1,0
13134,1,3,0,0,3,0,0,6,1,0,...,1,0,0,1,0,0,1,1,0,0


In [13]:
#clearing memory
del(train_lectures)

Affirmatives (True) for content_type_id are only for those with a different type of content (lectures). These are not real questions.

In [14]:
#removing True or 1 for content_type_id

train = train[train.content_type_id == False].sort_values('timestamp').reset_index(drop = True)

In [15]:
train[(train.task_container_id == 9999)].tail()

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
1786449,24925609370,39919444,7979,0,9999,1,19000.0,True
1847134,31352567848,40224694,5316,0,9999,1,8000.0,True


In [16]:
train[(train.content_type_id == False)].task_container_id.nunique()

9999

In [17]:
#saving value to fillna
elapsed_mean = train.prior_question_elapsed_time.mean()


In [18]:
group1 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['count'])
group1.columns = ['avg_questions']
group2 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['nunique'])
group2.columns = ['avg_questions']
group3 = group1 / group2

In [19]:
group3['avg_questions_seen'] = group3.avg_questions.cumsum()

In [20]:
group3.iloc[0].avg_questions_seen

1.0062272963155163

In [21]:
results_u_final = train.loc[train.content_type_id == False, ['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_final.columns = ['answered_correctly_user']

results_u2_final = train.loc[train.content_type_id == False, ['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
results_u2_final.columns = ['explanation_mean_user']

In [22]:
results_u2_final.explanation_mean_user.describe()

count    7709.000000
mean        0.565149
std         0.360969
min         0.000000
25%         0.230769
50%         0.650000
75%         0.906061
max         1.000000
Name: explanation_mean_user, dtype: float64

In [23]:
train = pd.merge(train, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

In [24]:
results_q_final = train.loc[train.content_type_id == False, ['question_id','answered_correctly']].groupby(['question_id']).agg(['mean'])
results_q_final.columns = ['quest_pct']

In [25]:
results_q2_final = train.loc[train.content_type_id == False, ['question_id','part']].groupby(['question_id']).agg(['count'])
results_q2_final.columns = ['count']

In [26]:
question2 = pd.merge(questions_df, results_q_final, left_on = 'question_id', right_on = 'question_id', how = 'left')

In [27]:
question2 = pd.merge(question2, results_q2_final, left_on = 'question_id', right_on = 'question_id', how = 'left')

In [28]:
question2.quest_pct = round(question2.quest_pct,5)

In [29]:
display(question2.head(), question2.tail())

,question_id,part,quest_pct,count
0,0,1,0.89474,152.0
1,1,1,0.92667,150.0
2,2,1,0.55386,919.0
3,3,1,0.79452,438.0
4,4,1,0.64154,650.0


,question_id,part,quest_pct,count
13518,13518,5,0.76471,17.0
13519,13519,5,0.47059,17.0
13520,13520,5,0.70588,17.0
13521,13521,5,0.66667,21.0
13522,13522,5,0.82353,17.0


In [30]:
train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,part
0,0,115,5692,0,1,1,NaN,<NA>,5692,5
1,0,39068636,7900,0,0,1,NaN,<NA>,7900,1
2,0,2121898,5556,0,0,1,NaN,<NA>,5556,5
3,0,11460655,128,0,0,1,NaN,<NA>,128,1
4,0,11459815,7900,0,0,1,NaN,<NA>,7900,1


In [31]:
len(train)

1961017

## Data Exploration ##

In [32]:
len(train)

1961017

In [33]:
train.answered_correctly.mean()

0.6544859121568044

In [34]:
prior_mean_user = results_u2_final.explanation_mean_user.mean()

In [35]:
train.loc[(train.timestamp == 0)].answered_correctly.mean()

0.6812790097988654

In [36]:
train.loc[(train.timestamp != 0)].answered_correctly.mean()

0.654379522245107

In [37]:
train.drop(['timestamp', 'content_type_id', 'question_id', 'part'], axis=1, inplace=True)

In [38]:
len(train)

1961017

## Creating Validation Set (Most Recent Answers by User) ##

In [39]:
validation = train.groupby('user_id').tail(5)
train = train[~train.index.isin(validation.index)]
len(train) + len(validation)

1961017

In [40]:
validation.answered_correctly.mean()

0.5400544959128065

In [41]:
train.answered_correctly.mean()

0.6567796213436589

In [42]:
results_u_val = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_val.columns = ['answered_correctly_user']

results_u2_val = train[['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
results_u2_val.columns = ['explanation_mean_user']

Does it make sense to use last questions as validation? Why is the rate of correct answers so low?
I am convinced there is a better way to match the test data.

## Extracting Training Data ##

In [43]:
X = train.groupby('user_id').tail(18)
train = train[~train.index.isin(X.index)]
len(X) + len(train) + len(validation)

1961017

In [44]:
X.answered_correctly.mean()

0.5514405200868873

In [45]:
train.answered_correctly.mean()

0.6643792185571621

In [46]:
results_u_X = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_X.columns = ['answered_correctly_user']

results_u2_X = train[['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
results_u2_X.columns = ['explanation_mean_user']

## Merging Data ##

In [47]:
#clearing memory
del(train)

In [48]:
X = pd.merge(X, group3, left_on=['task_container_id'], right_index= True, how="left")
X = pd.merge(X, results_u_X, on=['user_id'], how="left")
X = pd.merge(X, results_u2_X, on=['user_id'], how="left")

X = pd.merge(X, user_lecture_stats_part, on=['user_id'], how="left")

In [49]:
validation = pd.merge(validation, group3, left_on=['task_container_id'], right_index= True, how="left")
validation = pd.merge(validation, results_u_val, on=['user_id'], how="left")
validation = pd.merge(validation, results_u2_val, on=['user_id'], how="left")

validation = pd.merge(validation, user_lecture_stats_part, on=['user_id'], how="left")

In [50]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X.prior_question_had_explanation.fillna(False, inplace = True)
validation.prior_question_had_explanation.fillna(False, inplace = True)

validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])

In [51]:
#reading in question df
#question2 = pd.read_csv('/kaggle/input/question2/question2.csv)

In [52]:
content_mean = question2.quest_pct.mean()

question2.quest_pct.mean()
#there are a lot of high percentage questions, should use median instead?

0.7022135696126812

In [53]:
#filling questions with no info with a new value
question2.quest_pct = question2.quest_pct.mask((question2['count'] < 3), .65)


#filling very hard new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct < .2) & (question2['count'] < 21), .2)

#filling very easy new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct > .95) & (question2['count'] < 21), .95)

In [54]:
X = pd.merge(X, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
validation = pd.merge(validation, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
X.part = X.part - 1
validation.part = validation.part - 1

In [55]:
X.head()

,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,avg_questions,avg_questions_seen,answered_correctly_user,explanation_mean_user,...,part_7_boolean,type_of_concept_boolean,type_of_intention_boolean,type_of_solving_question_boolean,type_of_starter_boolean,prior_question_had_explanation_enc,question_id,part,quest_pct,count
0,11450549,7900,0,1,NaN,False,1.006227,1.006227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,7900,0,0.81484,3559.0
1,27474736,4850,0,1,NaN,False,1.006227,1.006227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,4850,4,0.45679,81.0
2,27399024,4133,0,1,NaN,False,1.006227,1.006227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,4133,4,0.74815,135.0
3,27420468,5299,0,1,NaN,False,1.006227,1.006227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,5299,4,0.49074,216.0
4,19539567,5991,0,0,NaN,False,1.006227,1.006227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,5991,4,0.83938,193.0


In [56]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)
X.head()

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [57]:
X = X[['answered_correctly_user', 'explanation_mean_user', 'quest_pct', 'avg_questions_seen',
       'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part',
       'part_1', 'part_2', 'part_3', 'part_4', 'part_5', 'part_6', 'part_7',
       'type_of_concept', 'type_of_intention', 'type_of_solving_question', 'type_of_starter',
       'part_1_boolean', 'part_2_boolean', 'part_3_boolean', 'part_4_boolean', 'part_5_boolean', 'part_6_boolean', 'part_7_boolean',
       'type_of_concept_boolean', 'type_of_intention_boolean', 'type_of_solving_question_boolean', 'type_of_starter_boolean']]
X_val = X_val[['answered_correctly_user', 'explanation_mean_user', 'quest_pct', 'avg_questions_seen',
               'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part',
               'part_1', 'part_2', 'part_3', 'part_4', 'part_5', 'part_6', 'part_7',
               'type_of_concept', 'type_of_intention', 'type_of_solving_question', 'type_of_starter',
               'part_1_boolean', 'part_2_boolean', 'part_3_boolean', 'part_4_boolean', 'part_5_boolean', 'part_6_boolean', 'part_7_boolean',
               'type_of_concept_boolean', 'type_of_intention_boolean', 'type_of_solving_question_boolean', 'type_of_starter_boolean']]

In [58]:

# Filling with 0.5 for simplicity; there could likely be a better value
X['answered_correctly_user'].fillna(0.65,  inplace=True)
X['explanation_mean_user'].fillna(prior_mean_user,  inplace=True)
X['quest_pct'].fillna(content_mean, inplace=True)

X['part'].fillna(4, inplace = True)
X['avg_questions_seen'].fillna(1, inplace = True)
X['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
X['prior_question_had_explanation_enc'].fillna(0, inplace = True)

X['part_1'].fillna(0, inplace = True)
X['part_2'].fillna(0, inplace = True)
X['part_3'].fillna(0, inplace = True)
X['part_4'].fillna(0, inplace = True)
X['part_5'].fillna(0, inplace = True)
X['part_6'].fillna(0, inplace = True)
X['part_7'].fillna(0, inplace = True)
X['type_of_concept'].fillna(0, inplace = True)
X['type_of_intention'].fillna(0, inplace = True)
X['type_of_solving_question'].fillna(0, inplace = True)
X['type_of_starter'].fillna(0, inplace = True)
X['part_1_boolean'].fillna(0, inplace = True)
X['part_2_boolean'].fillna(0, inplace = True)
X['part_3_boolean'].fillna(0, inplace = True)
X['part_4_boolean'].fillna(0, inplace = True)
X['part_5_boolean'].fillna(0, inplace = True)
X['part_6_boolean'].fillna(0, inplace = True)
X['part_7_boolean'].fillna(0, inplace = True)
X['type_of_concept_boolean'].fillna(0, inplace = True)
X['type_of_intention_boolean'].fillna(0, inplace = True)
X['type_of_solving_question_boolean'].fillna(0, inplace = True)
X['type_of_starter_boolean'].fillna(0, inplace = True)

In [59]:
X_val['answered_correctly_user'].fillna(0.65,  inplace=True)
X_val['explanation_mean_user'].fillna(prior_mean_user,  inplace=True)
X_val['quest_pct'].fillna(content_mean,  inplace=True)

X_val['part'].fillna(4, inplace = True)
X_val['avg_questions_seen'].fillna(1, inplace = True)
X_val['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
X_val['prior_question_had_explanation_enc'].fillna(0, inplace = True)

X_val['part_1'].fillna(0, inplace = True)
X_val['part_2'].fillna(0, inplace = True)
X_val['part_3'].fillna(0, inplace = True)
X_val['part_4'].fillna(0, inplace = True)
X_val['part_5'].fillna(0, inplace = True)
X_val['part_6'].fillna(0, inplace = True)
X_val['part_7'].fillna(0, inplace = True)
X_val['type_of_concept'].fillna(0, inplace = True)
X_val['type_of_intention'].fillna(0, inplace = True)
X_val['type_of_solving_question'].fillna(0, inplace = True)
X_val['type_of_starter'].fillna(0, inplace = True)
X_val['part_1_boolean'].fillna(0, inplace = True)
X_val['part_2_boolean'].fillna(0, inplace = True)
X_val['part_3_boolean'].fillna(0, inplace = True)
X_val['part_4_boolean'].fillna(0, inplace = True)
X_val['part_5_boolean'].fillna(0, inplace = True)
X_val['part_6_boolean'].fillna(0, inplace = True)
X_val['part_7_boolean'].fillna(0, inplace = True)
X_val['type_of_concept_boolean'].fillna(0, inplace = True)
X_val['type_of_intention_boolean'].fillna(0, inplace = True)
X_val['type_of_solving_question_boolean'].fillna(0, inplace = True)
X_val['type_of_starter_boolean'].fillna(0, inplace = True)

## Modeling ##

In [60]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X_val = scaler.transform(X_val)
X_train = X.reshape(X.shape[0], X.shape[1])
X_test = X_val.reshape(X_val.shape[0], X_val.shape[1])

In [61]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.metrics import *
from torch.utils.data import DataLoader
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization,Dropout,Dense,Flatten,Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
import time

try:
    from tensorflow.python.keras.callbacks import CallbackList
except ImportError:
    from tensorflow.python.keras._impl.keras.callbacks import CallbackList
    
def slice_arrays(arrays, start=None, stop=None):
    """Slice an array or list of arrays.

    This takes an array-like, or a list of
    array-likes, and outputs:
        - arrays[start:stop] if `arrays` is an array-like
        - [x[start:stop] for x in arrays] if `arrays` is a list

    Can also work on list/array of indices: `slice_arrays(x, indices)`

    Arguments:
        arrays: Single array or list of arrays.
        start: can be an integer index (start index)
            or a list/array of indices
        stop: integer (stop index); should be None if
            `start` was a list.

    Returns:
        A slice of the array(s).

    Raises:
        ValueError: If the value of start is a list and stop is not None.
    """

    if arrays is None:
        return [None]

    if isinstance(arrays, np.ndarray):
        arrays = [arrays]

    if isinstance(start, list) and stop is not None:
        raise ValueError('The stop argument has to be None if the value of start '
                         'is a list.')
    elif isinstance(arrays, list):
        if hasattr(start, '__len__'):
            # hdf5 datasets only support list objects as indices
            if hasattr(start, 'shape'):
                start = start.tolist()
            return [None if x is None else x[start] for x in arrays]
        else:
            if len(arrays) == 1:
                return arrays[0][start:stop]
            return [None if x is None else x[start:stop] for x in arrays]
    else:
        if hasattr(start, '__len__'):
            if hasattr(start, 'shape'):
                start = start.tolist()
            return arrays[start]
        elif hasattr(start, '__getitem__'):
            return arrays[start:stop]
        else:
            return [None]


class BaseModel(nn.Module):
    def __init__(self, device='cpu'):
        super(BaseModel, self).__init__()
        
        # device
        self.device = device 
        
        self.to(device)
    
    def compile(self, optimizer,
                loss=None,
                metrics=None,
                ):
        """
        :param optimizer: String (name of optimizer) or optimizer instance. See [optimizers](https://pytorch.org/docs/stable/optim.html).
        :param loss: String (name of objective function) or objective function. See [losses](https://pytorch.org/docs/stable/nn.functional.html#loss-functions).
        :param metrics: List of metrics to be evaluated by the model during training and testing. Typically you will use `metrics=['accuracy']`.
        """
        self.metrics_names = ["loss"]
        self.optim = self._get_optim(optimizer)
        self.loss_func = self._get_loss_func(loss)
        self.metrics = self._get_metrics(metrics)

    def _get_optim(self, optimizer):
        if isinstance(optimizer, str):
            if optimizer == "sgd":
                optim = torch.optim.SGD(self.parameters(), lr=0.01)
            elif optimizer == "adam":
                optim = torch.optim.Adam(self.parameters())  # 0.001
            elif optimizer == "adagrad":
                optim = torch.optim.Adagrad(self.parameters())  # 0.01
            elif optimizer == "rmsprop":
                optim = torch.optim.RMSprop(self.parameters())
            else:
                raise NotImplementedError
        else:
            optim = optimizer
        return optim

    def _get_loss_func(self, loss):
        if isinstance(loss, str):
            if loss == "binary_crossentropy":
                loss_func = F.binary_cross_entropy
            elif loss == "mse":
                loss_func = F.mse_loss
            elif loss == "mae":
                loss_func = F.l1_loss
            else:
                raise NotImplementedError
        else:
            loss_func = loss
        return loss_func

    def _log_loss(self, y_true, y_pred, eps=1e-7, normalize=True, sample_weight=None, labels=None):
        # change eps to improve calculation accuracy
        return log_loss(y_true,
                        y_pred,
                        eps,
                        normalize,
                        sample_weight,
                        labels)

    def _get_metrics(self, metrics, set_eps=False):
        metrics_ = {}
        if metrics:
            for metric in metrics:
                if metric == "binary_crossentropy" or metric == "logloss":
                    if set_eps:
                        metrics_[metric] = self._log_loss
                    else:
                        metrics_[metric] = log_loss
                if metric == "auc":
                    metrics_[metric] = roc_auc_score
                if metric == "mse":
                    metrics_[metric] = mean_squared_error
                if metric == "accuracy" or metric == "acc":
                    metrics_[metric] = lambda y_true, y_pred: accuracy_score(
                        y_true, np.where(y_pred > 0.5, 1, 0))
                self.metrics_names.append(metric)
        return metrics_
    
    def fit(self, x = None, y = None, batch_size=None, epochs=1, verbose=2,  validation_split=0.1, shuffle=True, callbacks=None):
        
        if validation_split and 0. < validation_split < 1.:
            do_validation = True
            
            split_at = int(x.shape[0] * (1. - validation_split))
                
            x_, val_x = x[0:split_at], x[split_at: -1]
            
            y_, val_y = y[0:split_at], y[split_at: -1]
            
            x = x_
            y = y_
        else:
            do_validation = False
            val_x = []
            val_y = []
        
        train_tensor_data = Data.TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
        
        if batch_size is None:
            batch_size = 256
            
        train_loader = DataLoader(
            dataset=train_tensor_data, shuffle=shuffle, batch_size=batch_size)
        
        print(self.device, end="\n")
        model = self.train()
        loss_func = self.loss_func
        optim = self.optim
        
        sample_num = len(train_tensor_data)
        steps_per_epoch = (sample_num - 1) // batch_size + 1

        callbacks = CallbackList(callbacks)
        callbacks.set_model(self)
        callbacks.on_train_begin()
        self.stop_training = False  # used for early stopping

        # Train
        print("Train on {0} samples, validate on {1} samples, {2} steps per epoch".format(
            len(train_tensor_data), len(val_y), steps_per_epoch))
        
        for epoch in range(epochs):
            callbacks.on_epoch_begin(epoch)
            epoch_logs = {}
            start_time = time.time()
            loss_epoch = 0
            total_loss_epoch = 0
            train_result = {}
            try:
                with tqdm(enumerate(train_loader), disable = verbose != 1) as t:
                    for index, (x_train, y_train) in t:
                        x = x_train.to(self.device).float()
                        y = y_train.to(self.device).float()

                        y_pred = model(x).squeeze()
                        #---------------------------
                        #print(y_pred.shape)
                        
                        optim.zero_grad()
                        loss = loss_func(y_pred, y.squeeze(), reduction='sum')
                        
                        #reg_loss = self.get_regularization_loss()

                        #total_loss = loss + reg_loss + self.aux_loss
                        total_loss = loss
                        
                        loss_epoch += loss.item()
                        total_loss_epoch += total_loss.item()
                        total_loss.backward(retain_graph=True)
                        
                        optim.step()

                        if verbose > 0:
                            for name, metric_fun in self.metrics.items():
                                if name not in train_result:
                                    train_result[name] = []
                                train_result[name].append(metric_fun(
                                    y.cpu().data.numpy(), y_pred.cpu().data.numpy().astype("float64")))

            except KeyboardInterrupt:
                t.close()
                raise
            t.close()

            # Add epoch_logs
            epoch_logs["loss"] = total_loss_epoch / sample_num
            for name, result in train_result.items():
                epoch_logs[name] = np.sum(result) / steps_per_epoch

            if do_validation:
                eval_result = self.evaluate(val_x, val_y, batch_size)
                for name, result in eval_result.items():
                    epoch_logs["val_" + name] = result
                    
            # verbose
            if verbose > 0:
                epoch_time = int(time.time() - start_time)
                print('Epoch {0}/{1}'.format(epoch + 1, epochs))

                eval_str = "{0}s - loss: {1: .4f}".format(
                    epoch_time, epoch_logs["loss"])

                for name in self.metrics:
                    eval_str += " - " + name + \
                                ": {0: .4f}".format(epoch_logs[name])

                if do_validation:
                    for name in self.metrics:
                        eval_str += " - " + "val_" + name + \
                                    ": {0: .4f}".format(epoch_logs["val_" + name])
                print(eval_str)
            callbacks.on_epoch_end(epoch, epoch_logs)
            if self.stop_training:
                break

        callbacks.on_train_end()
        
    def evaluate(self, x, y, batch_size=256):
        """

        :param x: Numpy array of test data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs).
        :param y: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs).
        :param batch_size: Integer or `None`. Number of samples per evaluation step. If unspecified, `batch_size` will default to 256.
        :return: Dict contains metric names and metric values.
        """
        pred_ans = self.predict(x, batch_size)
        eval_result = {}
        for name, metric_fun in self.metrics.items():
            eval_result[name] = metric_fun(y, pred_ans)
        return eval_result

    def predict(self, x, batch_size=256):
        """

        :param x: The input data, as a Numpy array (or list of Numpy arrays if the model has multiple inputs).
        :param batch_size: Integer. If unspecified, it will default to 256.
        :return: Numpy array(s) of predictions.
        """
        model = self.eval()
        tensor_data = Data.TensorDataset(torch.tensor(x))
        test_loader = DataLoader(
            dataset=tensor_data, shuffle=False, batch_size=batch_size)

        pred_ans = []
        with torch.no_grad():
            for index, x_test in enumerate(test_loader):
                x = x_test[0].to(self.device).float()

                y_pred = model(x).cpu().data.numpy()  # .squeeze()
                pred_ans.append(y_pred)

        return np.concatenate(pred_ans).astype("float64")


class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1, device = 'cpu'):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.to(device)

    def forward(self, q, k, v, mask=None):

        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)

        return output, attn
    
    
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim = 32, num_heads = 8, device = 'cpu'):
        super(MultiHeadAttention, self).__init__()
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
            
        self.projection_dim = embed_dim // num_heads
        
        self.query_dense = nn.Linear(embed_dim, embed_dim, bias=False)
        self.key_dense = nn.Linear(embed_dim, embed_dim, bias=False)
        self.value_dense = nn.Linear(embed_dim, embed_dim, bias=False)
        self.combine_heads = nn.Linear(embed_dim, embed_dim, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        
        self.to(device)
        
    def attention(self, query, key, value):
        score = torch.matmul(query, key.transpose(-1, -2))
        dim_key = torch.tensor(key.shape[-1], dtype = torch.float32)
        scaled_score = score / torch.sqrt(dim_key)
        weights = self.softmax(scaled_score)
        output = torch.matmul(weights, value)
        return output, weights
    
    def separate_heads(self, x, batch_size):
        x = x.reshape((batch_size, -1, self.num_heads, self.projection_dim))
        return x.permute(0,2,1,3)
    
    def forward(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = inputs.shape[0]
        
        # (batch_size, seq_len, embed_dim)
        query = self.query_dense(inputs) 
        
        # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  
        
        # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)
        
        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.separate_heads(
            query, batch_size
        )  
        
        # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  
        
         # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        ) 
        attention, weights = self.attention(query, key, value)
        
        # (batch_size, seq_len, num_heads, projection_dim)
        attention = attention.permute(0, 2, 1, 3) 
        
        # (batch_size, seq_len, embed_dim)
        concat_attention = attention.reshape(batch_size, -1, self.embed_dim)  
        
         # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(concat_attention) 
        return output


class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, device = 'cpu'):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = nn.Embedding(num_embeddings  = vocab_size, embedding_dim  = embed_dim)
        self.pos_emb = nn.Embedding(num_embeddings = maxlen, embedding_dim = embed_dim)
        self.device = device
        self.to(device)
    
    # 这里默认输入的是tensor
    def forward(self, x):
        maxlen = x.shape[-1]
        positions = torch.arange(0, maxlen).to(self.device)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    
class TransformerBlock(BaseModel):
    def __init__(self, embed_dim = 32 , num_heads = 8 , ff_dim = 64, dropout = 0.2, device = 'cuda:0'):
        super(TransformerBlock, self).__init__(device = device)
        self.embedding_layer = TokenAndPositionEmbedding(maxlen = 32, vocab_size = 50000, embed_dim = 32, device = device)

        self.att = MultiHeadAttention(embed_dim = 32, num_heads = 8, device = device)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(inplace = True),
            nn.Linear(ff_dim, embed_dim)
        )
        
        self.layernorm1 = nn.LayerNorm(embed_dim, eps = 1e-6)
        self.layernorm2 = nn.LayerNorm(embed_dim, eps = 1e-6)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout3 = nn.Dropout(dropout)
        
        self.linear1 = nn.Linear(embed_dim, 20)
        self.relu1 = nn.ReLU(inplace = True)
        
        self.dropout4 = nn.Dropout(dropout)
        
        self.outlinear = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.device = device
        self.to(device)

    def forward(self, inputs):
        #print(inputs.shape)
        inputs = self.embedding_layer(inputs.type(torch.long))
        attn_output = self.att(inputs)
        
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        
        out = self.layernorm2(out1 + ffn_output)
        out = self.global_avg_pool(out)
        out = torch.flatten(out, start_dim=1)
        
        out = self.dropout3(out)
        
        out = self.linear1(out)
        out = self.relu1(out)
        out = self.dropout4(out)
        
        outputs = self.sigmoid(self.outlinear(out))
        
        return outputs    

'''
#test -------------
input_ = np.array([1,2,3,4,5,6,7,8,9]).reshape(3,3)
input_ = keras.preprocessing.sequence.pad_sequences(input_, maxlen=32)
input_ = torch.from_numpy(input_).type(torch.long)
#print(input_.shape)
test = TokenAndPositionEmbedding(50000, 10, 32)
a = test(input_)
#print(a.shape)
test_att = TransformerBlock()
a = test_att(a)
print(a.shape)
'''

maxlen = 32 
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)


# Embedding size for each token
embed_dim = 32 

# Number of attention heads
num_heads = 8  

# Hidden layer size in feed forward network inside transformer
ff_dim = 64  

vocab_size = 50000

model = TransformerBlock(embed_dim, num_heads, ff_dim)
model.compile('adam', 'binary_crossentropy',metrics=["binary_crossentropy", "auc"],)
model.fit(X_train, y.values, batch_size=2**10, epochs=100, validation_split=0.1)

cuda:0
Train on 116426 samples, validate on 12936 samples, 114 steps per epoch
Epoch 1/100
1s - loss:  0.6888 - binary_crossentropy:  0.6888 - auc:  0.5306 - val_binary_crossentropy:  0.6580 - val_auc:  0.5038
Epoch 2/100
1s - loss:  0.6649 - binary_crossentropy:  0.6649 - auc:  0.6149 - val_binary_crossentropy:  0.6558 - val_auc:  0.5210
Epoch 3/100
1s - loss:  0.6645 - binary_crossentropy:  0.6645 - auc:  0.6170 - val_binary_crossentropy:  0.6557 - val_auc:  0.5281
Epoch 4/100
1s - loss:  0.6642 - binary_crossentropy:  0.6642 - auc:  0.6176 - val_binary_crossentropy:  0.6552 - val_auc:  0.5347
Epoch 5/100
1s - loss:  0.6639 - binary_crossentropy:  0.6639 - auc:  0.6191 - val_binary_crossentropy:  0.6547 - val_auc:  0.5269
Epoch 6/100
1s - loss:  0.6639 - binary_crossentropy:  0.6638 - auc:  0.6190 - val_binary_crossentropy:  0.6556 - val_auc:  0.5304
Epoch 7/100
1s - loss:  0.6638 - binary_crossentropy:  0.6638 - auc:  0.6195 - val_binary_crossentropy:  0.6548 - val_auc:  0.5304
Epoc

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
y_true = np.array(y_val)
roc_auc_score(y_true, y_pred)

## Making Predictions for New Data ##

In [ ]:
'''
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['task_container_id'] = test_df.task_container_id.mask(test_df.task_container_id > 9999, 9999)
    test_df = pd.merge(test_df, group3, left_on=['task_container_id'], right_index= True, how="left")
    test_df = pd.merge(test_df, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df = pd.merge(test_df, results_u_final, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_u2_final, on=['user_id'],  how="left")
    
    test_df = pd.merge(test_df, user_lecture_stats_part, on=['user_id'], how="left")
    test_df['part_1'].fillna(0, inplace = True)
    test_df['part_2'].fillna(0, inplace = True)
    test_df['part_3'].fillna(0, inplace = True)
    test_df['part_4'].fillna(0, inplace = True)
    test_df['part_5'].fillna(0, inplace = True)
    test_df['part_6'].fillna(0, inplace = True)
    test_df['part_7'].fillna(0, inplace = True)
    test_df['type_of_concept'].fillna(0, inplace = True)
    test_df['type_of_intention'].fillna(0, inplace = True)
    test_df['type_of_solving_question'].fillna(0, inplace = True)
    test_df['type_of_starter'].fillna(0, inplace = True)
    test_df['part_1_boolean'].fillna(0, inplace = True)
    test_df['part_2_boolean'].fillna(0, inplace = True)
    test_df['part_3_boolean'].fillna(0, inplace = True)
    test_df['part_4_boolean'].fillna(0, inplace = True)
    test_df['part_5_boolean'].fillna(0, inplace = True)
    test_df['part_6_boolean'].fillna(0, inplace = True)
    test_df['part_7_boolean'].fillna(0, inplace = True)
    test_df['type_of_concept_boolean'].fillna(0, inplace = True)
    test_df['type_of_intention_boolean'].fillna(0, inplace = True)
    test_df['type_of_solving_question_boolean'].fillna(0, inplace = True)
    test_df['type_of_starter_boolean'].fillna(0, inplace = True)
    
    test_df['answered_correctly_user'].fillna(0.65,  inplace=True)
    test_df['explanation_mean_user'].fillna(prior_mean_user,  inplace=True)
    test_df['quest_pct'].fillna(content_mean,  inplace=True)
    test_df['part'] = test_df.part - 1

    test_df['part'].fillna(4, inplace = True)
    test_df['avg_questions_seen'].fillna(1, inplace = True)
    test_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    X = test_df[['answered_correctly_user', 'explanation_mean_user', 'quest_pct', 'avg_questions_seen',
                                                            'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part',
                                                            'part_1', 'part_2', 'part_3', 'part_4', 'part_5', 'part_6', 'part_7',
                                                            'type_of_concept', 'type_of_intention', 'type_of_solving_question', 'type_of_starter',
                                                            'part_1_boolean', 'part_2_boolean', 'part_3_boolean', 'part_4_boolean', 'part_5_boolean', 'part_6_boolean', 'part_7_boolean',
                                                            'type_of_concept_boolean', 'type_of_intention_boolean', 'type_of_solving_question_boolean', 'type_of_starter_boolean']]
    X=scaler.transform(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=maxlen)
    test_df['answered_correctly'] =  model.predict(X)

    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
'''